In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3


engine = sqlite3.connect('example.db')
def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, engine)

In [ ]:
# SELECT *
# FROM Nations
# ORDER BY CountryCode
# OFFSET 100 ROWS
# FETCH NEXT 100 ROWS ONLY; 

In [105]:
df = pd.read_csv('./earthquake_1995-2023.csv')
df1 = pd.read_csv('./worldcities.csv')
# df = df[['iso2', 'city_ascii','population','lat', 'lng']]
# df.rename(columns={'iso2': 'CountryCode', 'city_ascii': 'CityName', 'population': 'Pop2017', 'lat': 'Latitude', 'lng': 'Longitude'}, inplace=True)
df['Date'] = pd.to_datetime(df['date_time'],dayfirst=True).dt.date
df['Time'] = pd.to_datetime(df['date_time'],dayfirst=True).dt.strftime('%H:%M:%S')
df.rename(columns={'date_time': 'DateTime', 'longitude': 'Longitude', 'latitude': 'Latitude', 'depth': 'Depth', 'magnitude': 'Magnitude', 'title': 'Title', 'country': 'Country'}, inplace=True)
df = df[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude', 'Title', 'Country','location']]
# [Date	Time	latitude	longitude	depth	magnitude	place	NearestPop	Country]]
   # ['title', 'magnitude', 'date_time', 'cdi', 'mmi', 'alert', 'tsunami',
    #    'sig', 'net', 'nst', 'dmin', 'gap', 'magType', 'depth', 'latitude',
    #    'longitude', 'location', 'continent', 'country'],
df['country'] = df['Country'].fillna(df['location'].str.split(',').str[-1].str.strip())
df['NearestPop'] = df['location'].str.split(',').str[0]
# df['Country'] = 
df = pd.merge(df,df1,right_on='city_ascii',left_on='NearestPop',how='left')[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude', 'location', 'NearestPop','iso2']]
df.rename(columns={'iso2': 'Country','location':'place'}, inplace=True)
df.drop_duplicates(subset=['place','Date','Time'],inplace=True)
df   
df = pd.read_clipboard()
df.to_sql('Orders', engine, if_exists='append', index=False)

100

In [ ]:
select PlayerName, Country,
round(Weight_kg/SQUARE(Height_cm/100),2) BMI
from Players Where Country = 'USA'
Or Country = 'Canada'
order by BMI;

SELECT PlayerName, Country,
ROUND(Weight_kg/SQUARE(Height_cm/100),2) BMI
FROM Players
WHERE Country = 'USA'
OR Country = 'Canada'
ORDER BY BMI;

In [4]:
rsq("""select PlayerName, Country,
round(Weight_kg/POWER(Height_cm/100, 2), 2) BMI
from Players Where Country = 'USA'
Or Country = 'Canada'
order by BMI;""")

,PlayerName,Country,BMI
0,Corey Brewer,USA,19.77
1,Dejounte Murray,USA,20.00
2,Brandon Ingram,USA,20.20
3,Kobi Simmons,USA,20.53
4,Terrance Ferguson,USA,20.56
...,...,...,...
434,Jameel Warney,USA,28.34
435,Nigel Hayes,USA,28.39
436,Jack Cooley,USA,28.42
437,Alan Williams,USA,28.88


In [7]:
rsq("""-- Your friend's query
-- First attempt, contains errors and inconsistent formatting
/*
select PlayerName, p.Country,sum(ps.TotalPoints)
AS TotalPoints
FROM PlayerStats ps inner join Players On ps.PlayerName = p.PlayerName
WHERE p.Country = 'New Zeeland'
Group
by PlayerName, Country
order by Country;
*/

-- Your query
-- Second attempt - errors corrected and formatting fixed

SELECT p.PlayerName, p.Country,
SUM(ps.TotalPoints) AS TotalPoints
FROM PlayerStats ps
INNER JOIN Players p
ON ps.PlayerName = p.PlayerName
WHERE p.Country = 'New Zealand'
GROUP BY p.PlayerName, p.Country;""")

,PlayerName,Country,TotalPoints
0,Steven Adams,New Zealand,1056


In [9]:
rsq("""SELECT p.PlayerName, p.Country,
SUM(ps.TotalPoints) AS TotalPoints
FROM PlayerStats ps
INNER JOIN Players p
ON ps.PlayerName = p.PlayerName
WHERE p.Country = 'Australia'
GROUP BY p.PlayerName, p.Country""")

,PlayerName,Country,TotalPoints
0,Andrew Bogut,Australia,36
1,Aron Baynes,Australia,482
2,Ben Simmons,Australia,1279
3,Dante Exum,Australia,114
4,Joe Ingles,Australia,940
5,Kyrie Irving,Australia,1466
6,Mangok Mathiang,Australia,8
7,Matthew Dellavedova,Australia,164
8,Patty Mills,Australia,819


In [ ]:
SELECT Team,
ROUND(AVG(BMI),2) AS AvgTeamBMI -- Alias the new column
FROM PlayerStats AS ps -- Alias PlayerStats table
INNER JOIN
(SELECT PlayerName, Country,
Weight_kg/SQUARE(Height_cm/100) BMI
FROM Players) AS p -- Alias the sub-query
-- Alias the joining columns
ON p.PlayerName = ps.PlayerName
GROUP BY Team
HAVING AVG(BMI) >= 25;

In [116]:
rsq("""SELECT Team,
ROUND(AVG(BMI),2) AS AvgTeamBMI -- Alias the new column
FROM PlayerStats AS ps -- Alias PlayerStats table
INNER JOIN
(SELECT PlayerName, Country,
Weight_kg/POWER(Height_cm/100, 2) BMI
FROM Players) AS p -- Alias the sub-query
-- Alias the joining columns
ON p.PlayerName = ps.PlayerName
GROUP BY Team
HAVING AVG(BMI) >= 25;""")

,Team,AvgTeamBMI
0,BOS,25.08
1,CLE,25.08
2,IND,25.02


In [115]:
rsq("""SELECT date, place, Depth, magnitude
FROM Earthquakes
WHERE magnitude > 6
    AND Depth > 100
ORDER BY Depth DESC;""")


,Date,place,Depth,Magnitude
0,2018-09-06,"Levuka, Fiji",670.810,7.9
1,2015-05-30,"Bonin Islands, Japan region",664.000,7.8
2,2022-11-09,the Fiji Islands,660.000,7.0
3,2022-11-09,the Fiji Islands,630.379,6.8
4,2018-08-24,"Iñapari, Peru",630.000,7.1
...,...,...,...,...
181,2023-01-09,"Pulau Pulau Tanimbar, Indonesia",105.249,7.6
182,2004-10-08,"Talisay, Philippines",105.000,6.5
183,2001-01-09,"Port-Olry, Vanuatu",103.000,7.1
184,2014-08-24,"Aniso, Peru",101.000,6.8


In [114]:
rsq("""SELECT -- Limit the number of rows to ten
*
FROM Earthquakes
WHERE country = 'PG'
OR Country = 'ID'
ORDER BY Depth; -- Order results from shallowest to deepest""")

,Date,Time,Latitude,Longitude,Depth,Magnitude,place,NearestPop,Country
0,2009-10-01,01:52:00,-2.4820,101.524,9.000,6.6,"Sungai Penuh, Indonesia",Sungai Penuh,ID
1,2013-04-23,23:14:00,-3.8980,152.127,10.000,6.5,"Rabaul, Papua New Guinea",Rabaul,PG
2,2004-11-26,02:25:00,-3.6090,135.404,10.000,7.1,"Nabire, Indonesia",Nabire,ID
3,2002-10-10,12:28:00,-1.5110,133.973,10.000,6.7,"Manokwari, Indonesia",Manokwari,ID
4,2002-10-10,10:50:00,-1.7570,134.297,10.000,7.6,"Manokwari, Indonesia",Manokwari,ID
...,...,...,...,...,...,...,...,...,...
138,2015-02-27,13:45:00,-7.2968,122.535,552.060,7.0,"Maumere, Indonesia",Maumere,ID
139,2017-10-24,10:47:00,-7.2168,123.074,553.810,6.7,"Maumere, Indonesia",Maumere,ID
140,1996-06-17,11:22:00,-7.1370,122.589,587.300,7.9,"Maumere, Indonesia",Maumere,ID
141,2023-04-14,09:55:00,-6.0255,112.033,594.028,7.0,"Tuban, Indonesia",Tuban,ID


In [112]:
rsq("""SELECT Date, Country, Place, Depth, Magnitude
FROM Earthquakes
WHERE Magnitude > 8
AND Depth > 500
ORDER BY Depth DESC;""")

,Date,Country,place,Depth,Magnitude
0,2018-08-19,None,"Levuka, Fiji",600.0,8.2
1,2013-05-24,RU,Okhotsk,598.1,8.3


In [110]:
rsq("""/*
Returns earthquakes in New Zealand with a magnitude of 7.5 or more
*/

SELECT Date, Place, NearestPop, Magnitude
FROM Earthquakes
WHERE Country = 'NZ'
AND Magnitude >= 7.5
ORDER BY Magnitude DESC;""")

,Date,place,NearestPop,Magnitude
0,2009-07-15,"Te Anau, New Zealand",Te Anau,7.8


In [25]:
rsq("""SELECT PlayerName,
Team,
Position,
(OffRebound+DefRebound)/CAST(GamesPlayed AS numeric) AS AvgRebounds
FROM PlayerStats
WHERE AvgRebounds >= 12;""")

,PlayerName,Team,Position,AvgRebounds
0,Dwight Howard,CHO,C,12
1,Andre Drummond,DET,C,15
2,DeAndre Jordan,LAC,C,15
3,Karl-Anthony Towns,MIN,C,12
4,DeMarcus Cousins,NOP,C,12


In [26]:
rsq("""SELECT PlayerName,
Country,
College,
DraftYear,
DraftNumber
FROM Players
-- WHERE UPPER(LEFT(College,5)) LIKE 'LOU%'
WHERE College LIKE 'Louisiana%'; -- Add the wildcard filter""")

,PlayerName,Country,College,DraftYear,DraftNumber
0,Antonio Blakeney,USA,Louisiana State,NaN,NaN
1,Ben Simmons,Australia,Louisiana State,2016.0,1.0
2,Elfrid Payton,USA,Louisiana-Lafayette,2014.0,10.0
3,Erik McCree,USA,Louisiana Tech,NaN,NaN
4,Garrett Temple,USA,Louisiana State,NaN,NaN
5,Jarell Martin,USA,Louisiana State,2015.0,25.0
6,Johnny O'Bryant,USA,Louisiana State,2014.0,36.0
7,Jordan Mickey,USA,Louisiana State,2015.0,33.0
8,Paul Millsap,USA,Louisiana Tech,2006.0,47.0
9,Tim Quarterman,USA,Louisiana State,NaN,NaN


In [106]:
rsq("""SELECT Country, COUNT(*) CountOfPlayers
FROM Players
GROUP BY Country
HAVING Country
IN ('Argentina','Brazil','Dominican Republic'
,'Puerto Rico');""")

,Country,CountOfPlayers
0,Argentina,2
1,Brazil,5
2,Dominican Republic,3
3,Puerto Rico,2
